<a href="https://colab.research.google.com/github/gracekazmierski/machinelearning/blob/main/bikerentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

##Major COVID-19 Restriction and Lockdown Periods in Beijing

###Early COVID-19 Pandemic (Early 2020)
####Start:
On January 23, 2020, as Wuhan went into lockdown, strict travel restrictions and prevention measures began across China. Beijing also implemented inter-city travel controls, closed public facilities, and recommended working from home.
####End:
In early April 2020, with the official lifting of Wuhan's lockdown, high-intensity national restrictions gradually eased. Beijing also began to normalize around this time.

###Spring 2022 (Omicron Variant Spread)
####Start:
In late April 2022, as Omicron variant cases surged in Beijing, severe restrictions resumed. April 25, 2022, marked the start of large-scale nucleic acid testing, accompanied by widespread measures such as residential compound lockdowns, bans on dine-in services, school closures, and the suspension of some subway stations.

####End:
In early June 2022, as the number of cases declined, Beijing authorities began lifting most lockdown measures and promoting normalization starting June 6, 2022. However, partial lockdowns of specific areas continued if sporadic cases emerged.

###Autumn 2022 (Just Before Zero-COVID Policy Repeal)
####Start:
From late October through November 2022, the number of cases surged in Beijing and other major Chinese cities, reaching the peak of the 'Zero-COVID' policy's intensity. Specific districts in Beijing, like Chaoyang District, experienced controls akin to full lockdowns. While the exact start date varied by region, these controls significantly intensified around mid-November 2022.
####End:
The Chinese government officially announced the repeal of the nationwide 'Zero-COVID' policy on December 7, 2022, effectively ending Beijing's severe restrictions. Although the number of cases surged thereafter, policy-wise, mandatory lockdowns and mass testing were eliminated.

In [ ]:
# --- Load the Data ---
try:
    df = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')
    print("bikes.csv file has been loaded.")
except FileNotFoundError:
    print("The file could not be found.")
    exit()

# 'dteday' to datetime
df['dteday'] = pd.to_datetime(df['dteday'])

# Create total_rentals column (casual + registered)
df['total_rentals'] = df['casual'] + df['registered']

# --- Feature Engineering  ---

# 1. Features about Time
df['year'] = df['dteday'].dt.year
df['month'] = df['dteday'].dt.month
df['day_of_week'] = df['dteday'].dt.dayofweek # Monday=0, Sunday=6

# 2. Create avg_temp_c column (temp_c + feels_like_c)
df['avg_temp_c'] = (df['temp_c'] + df['feels_like_c']) / 2

# 3. COVID-19 period

# 3-1. First covid
df['is_early_covid_2020'] = 0
df.loc[(df['dteday'] >= pd.to_datetime('2020-01-23')) &
       (df['dteday'] <= pd.to_datetime('2020-04-05')), 'is_early_covid_2020'] = 1

# 3-2. Omicron(Lockdown and ban public transportation)
df['is_spring_2022_omicron'] = 0
df.loc[(df['dteday'] >= pd.to_datetime('2022-04-25')) &
       (df['dteday'] <= pd.to_datetime('2022-06-05')), 'is_spring_2022_omicron'] = 1

# 3-3. Zero-covid in Beijing (Lockdown and ban public transportation)
df['is_autumn_2022_zero_covid'] = 0
df.loc[(df['dteday'] >= pd.to_datetime('2022-10-20')) &
       (df['dteday'] <= pd.to_datetime('2022-12-06')), 'is_autumn_2022_zero_covid'] = 1

# 4. Commute time (AM 7-9, PM 4-6)
df['is_commute_hour'] = 0
df.loc[((df['hr'] >= 7) & (df['hr'] <= 9)) |
       ((df['hr'] >= 16) & (df['hr'] <= 18)), 'is_commute_hour'] = 1

# 5. Weather + Hr
df['weathersit_hr_interaction'] = df['weathersit'].astype(str) + '_' + df['hr'].astype(str)


# Target
features = ['season', 'hr', 'holiday', 'workingday', 'weathersit',
            'avg_temp_c', 'hum', 'windspeed',
            'year', 'month', 'day_of_week',
            'is_early_covid_2020', 'is_spring_2022_omicron', 'is_autumn_2022_zero_covid',
            'is_commute_hour',
            'weathersit_hr_interaction']
target = 'total_rentals'

# categorical and numerical features setting
categorical_features = ['season', 'hr', 'holiday', 'workingday', 'weathersit',
                        'month', 'day_of_week', 'year',
                        'is_early_covid_2020', 'is_spring_2022_omicron', 'is_autumn_2022_zero_covid',
                        'is_commute_hour',
                        'weathersit_hr_interaction']
numerical_features = ['avg_temp_c', 'hum', 'windspeed']






bikes.csv file has been loaded.


In [ ]:
# prepocessing Pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# --- Data split ---
# Train set: 2011 to August 2023
train_df = df[(df['dteday'].dt.year < 2023) |
              ((df['dteday'].dt.year == 2023) & (df['dteday'].dt.month <= 8))]
# Internal Verification/Test Set: September 2023 to October 2023
test_df = df[(df['dteday'].dt.year == 2023) & (df['dteday'].dt.month > 8)]

X_train = train_df[features]
y_train = train_df[target]
X_test = test_df[features] # Internal Verification/Test Set
y_test = test_df[target]

print(f"\nTain data Scale: {X_train.shape[0]}")
print(f"Internal Verification/Test Set Scale (2023.09 - 2023.10): {X_test.shape[0]}")

# Preprocessor training and conversion (fit only with training data)
X_train_processed = preprocessor.fit_transform(X_train)
# Internal verification/test data is only transformed with trained preprocessor
X_test_processed = preprocessor.transform(X_test)

print(f"\nTraining data shape after preprocessing: {X_train_processed.shape}")
print(f"After preprocessing, internal verification/test data shape: {X_test_processed.shape}")

# Size of the input layer
input_shape = X_train_processed.shape[1]


Tain data Scale: 111011
Internal Verification/Test Set Scale (2023.09 - 2023.10): 1464

Training data shape after preprocessing: (111011, 175)
After preprocessing, internal verification/test data shape: (1464, 175)


In [ ]:
# --- Hyperparameter tuning with Early Stopping ---

# Define the hyperparameter grid to explore
learning_rates = [0.001, 0.0005]
layer_configs = [
    [128, 64, 32],      # Default
    [256, 128, 64],     # Larger
    [64, 32]          # Smaller
]
dropout_rates = [0.1, 0.3]

best_mae = float('inf')
best_params = {}
best_model = None
best_predictions_test = None
best_y_test = None

print("\nStart tuning hyperparameters (Early Stopping applied)...")
tune_results = []

# EarlyStopping Callback
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)



In [ ]:
for lr in learning_rates:
    for config in layer_configs:
        for drop_rate in dropout_rates:
            tf.keras.backend.clear_session() # Initialize sessions before training each model

            # Build a model
            model_tuned = keras.Sequential([
                layers.Input(shape=(input_shape,))
            ])
            for neurons in config:
                model_tuned.add(layers.Dense(neurons, activation='relu'))
                model_tuned.add(layers.Dropout(drop_rate))
            model_tuned.add(layers.Dense(1)) # Output layer

            # Compilation of models
            optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
            model_tuned.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

            # Model training (Early Stopping applied)
            history = model_tuned.fit(X_train_processed, y_train,
                                      epochs=200, # Set a sufficiently large number of epochs (early stopping early)
                                      batch_size=32,
                                      validation_data=(X_test_processed, y_test), # Monitor val_loss with internal validation/test set
                                      callbacks=[early_stopping_callback], # Add callback
                                      verbose=1)

            # Evaluation of the test dataset
            loss, mae = model_tuned.evaluate(X_test_processed, y_test, verbose=0)
            predictions_test = model_tuned.predict(X_test_processed).flatten()
            predictions_test = np.maximum(0, predictions_test) # Negative # = 0

            # Calculate additional indicators (R^2, Within %)
            r2 = r2_score(y_test, predictions_test)
            rmse = np.sqrt(mean_squared_error(y_test, predictions_test))
            median_abs_error = np.median(np.abs(y_test - predictions_test))

            # Division by zero prevention for cases where y_test is 0
            abs_errors_ratio = np.abs(y_test - predictions_test) / np.maximum(y_test, 1e-6)

            within_5_percent = np.mean(abs_errors_ratio * 100 <= 5) * 100
            within_10_percent = np.mean(abs_errors_ratio * 100 <= 10) * 100
            within_20_percent = np.mean(abs_errors_ratio * 100 <= 20) * 100


            print(f"LR: {lr:.5f}, Layers: {config}, Dropout: {drop_rate:.1f} -> MAE: {mae:.2f}, R2: {r2:.4f}, Within 20%: {within_20_percent:.2f}%")

            tune_results.append({
                'learning_rate': lr,
                'layer_config': config,
                'dropout_rate': drop_rate,
                'mae': mae,
                'rmse': rmse,
                'r2': r2,
                'within_5_percent': within_5_percent,
                'within_10_percent': within_10_percent,
                'within_20_percent': within_20_percent
            })

            # Best Model Update (based on MAE)
            if mae < best_mae:
                best_mae = mae
                best_params = {'learning_rate': lr, 'layer_config': config, 'dropout_rate': drop_rate}
                best_model = model_tuned
                best_predictions_test = predictions_test
                best_y_test = y_test # Save the actual y_test value

print("\nHyperparameter tuning completed.")
print(f"\nOptimal hyperparameters: {best_params}")
print(f"Optimal MAE: {best_mae:.2f}")

Epoch 1/200
 867/3470 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 104598.1328 - mae: 216.0402

KeyboardInterrupt: 

In [ ]:
# Output final performance metrics for optimal models
print("\nInternal Validation/Test Dataset of Optimal Models (September, October 2023) Final Performance:")
final_r2 = r2_score(best_y_test, best_predictions_test)
final_rmse = np.sqrt(mean_squared_error(best_y_test, best_predictions_test))
final_mae = mean_absolute_error(best_y_test, best_predictions_test)
final_median_abs_error = np.median(np.abs(best_y_test - best_predictions_test))

# Division by zero prevention when y_test is 0 (also applied to final performance calculations)
abs_errors_ratio_final = np.abs(best_y_test - best_predictions_test) / np.maximum(best_y_test, 1e-6)

final_within_5_percent = np.mean(abs_errors_ratio_final * 100 <= 5) * 100
final_within_10_percent = np.mean(abs_errors_ratio_final * 100 <= 10) * 100
final_within_20_percent = np.mean(abs_errors_ratio_final * 100 <= 20) * 100

print(f"R^2: {final_r2:.4f}")
print(f"RMSE: {final_rmse:.2f}")
print(f"Mean Absolute Error: {final_mae:.2f}")
print(f"Median Absolute Error: {final_median_abs_error:.2f}")
print(f"Within 5%: {final_within_5_percent:.2f}%")
print(f"Within 10%: {final_within_10_percent:.2f}%")
print(f"Within 20%: {final_within_20_percent:.2f}%")


Internal Validation/Test Dataset of Optimal Models (September, October 2023) Final Performance:
R^2: 0.9598
RMSE: 100.41
Mean Absolute Error: 68.03
Median Absolute Error: 42.94
Within 5%: 26.30%
Within 10%: 50.55%
Within 20%: 77.46%


In [ ]:
# --- Save the optimal model ---
model_save_path = 'best_bike_rental_model.keras'
if best_model is not None:
    best_model.save(model_save_path)
    print(f"\nThe Best model has been saved to '{model_save_path}'.")
else:
    print("\nUnable to save because the optimal model was not found.")


The Best model has been saved to 'best_bike_rental_model.keras'.


In [ ]:
model_load_path = 'best_bike_rental_model.keras'

try:
    # load the model
    loaded_model = tf.keras.models.load_model(model_load_path)
    print(f"The model has been successfully laoded from '{model_load_path}'.")

    # Output summary information of the loaded model (make sure the model is loaded properly)
    loaded_model.summary()

except Exception as e:
    print(f"An error occurred while loading the model.: {e}")
    print("Please check the file path, or verify that the model is stored correctly in that path.")

The model has been successfully laoded from 'best_bike_rental_model.keras'.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        45,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 258,821 (1011.02 KB)

 Trainable params: 86,273 (337.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 172,548 (674.02 KB)

In [ ]:
# --- Predictions and file storage for holdout datasets (using optimal models) ---


print("\nLoad holdout dataset and start prediction...")

try:
    # biking_holdout_test_mini.csv load
    holdout_df = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/biking_holdout_test_mini.csv')
    print("biking_holdout_test_mini.csv has been successfully loaded")
except FileNotFoundError:
    print("biking_holdout_test_mini.csv could not be found.")
    exit()

# Convert 'dteday' column to datetime format (also applies to holdout_df)
holdout_df['dteday'] = pd.to_datetime(holdout_df['dteday'])

# Add features to the holdout dataset (apply new features added to the df equally to holdout_df)
holdout_df['year'] = holdout_df['dteday'].dt.year
holdout_df['month'] = holdout_df['dteday'].dt.month
holdout_df['day_of_week'] = holdout_df['dteday'].dt.dayofweek

# Avg_temp_c
holdout_df['avg_temp_c'] = (holdout_df['temp_c'] + holdout_df['feels_like_c']) / 2

# 1. COVID-19
holdout_df['is_early_covid_2020'] = 0
holdout_df.loc[(holdout_df['dteday'] >= pd.to_datetime('2020-01-23')) &
               (holdout_df['dteday'] <= pd.to_datetime('2020-04-05')), 'is_early_covid_2020'] = 1

holdout_df['is_spring_2022_omicron'] = 0
holdout_df.loc[(holdout_df['dteday'] >= pd.to_datetime('2022-04-25')) &
               (holdout_df['dteday'] <= pd.to_datetime('2022-06-05')), 'is_spring_2022_omicron'] = 1

holdout_df['is_autumn_2022_zero_covid'] = 0
holdout_df.loc[(holdout_df['dteday'] >= pd.to_datetime('2022-10-20')) &
               (holdout_df['dteday'] <= pd.to_datetime('2022-12-06')), 'is_autumn_2022_zero_covid'] = 1

# 2. commute time
holdout_df['is_commute_hour'] = 0
holdout_df.loc[((holdout_df['hr'] >= 7) & (holdout_df['hr'] <= 9)) |
               ((holdout_df['hr'] >= 16) & (holdout_df['hr'] <= 18)), 'is_commute_hour'] = 1

# 3. weather + hr
holdout_df['weathersit_hr_interaction'] = holdout_df['weathersit'].astype(str) + '_' + holdout_df['hr'].astype(str)

# extract the features
X_holdout = holdout_df[features]

print(f"\nholdout dataset scale: {X_holdout.shape[0]}")

# Transform the preprocessed holdout dataset (using transform instead of fit_transform)
X_holdout_processed = preprocessor.transform(X_holdout)

print(f"After preprocessing holdout dataset Shape: {X_holdout_processed.shape}")

# Performing predictions on the holdout dataset
holdout_predictions = best_model.predict(X_holdout_processed).flatten()

# Because the prediction may be negative, treat less than 0 as 0
holdout_predictions = np.maximum(0, holdout_predictions)

# Create DataFrame to contain predictive results and set column names to meet requirements
predictions_output_df = pd.DataFrame(holdout_predictions, columns=['predictions'])

# Set the file name (team4-module4-predictions.csv)
output_filename_holdout = 'team4-module4-predictions.csv'

# Save to csv file
predictions_output_df.to_csv(output_filename_holdout, index=False)

print(f"\nholdout dataset's predictions has been saved to '{output_filename_holdout}'.")
print("\nPreview holdout prediction results (top five):")
print(predictions_output_df.head())


Load holdout dataset and start prediction...
biking_holdout_test_mini.csv has been successfully loaded

holdout dataset scale: 384
After preprocessing holdout dataset Shape: (384, 175)
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 

holdout dataset's predictions has been saved to 'team4-module4-predictions.csv'.

Preview holdout prediction results (top five):
   predictions
0    72.510139
1    38.069267
2    16.789047
3    10.851306
4    15.733383
